In [1]:
### code to generate modified Probability Model (Prob_model.csv)

In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [3]:
clean_data=pd.read_csv('newtb2.csv')

In [4]:
clean_data=clean_data.replace('N', 0)
zr=np.zeros((1,len(clean_data)))[0]
zr[clean_data['Structural improvement Needed']!=0]=1
clean_data['need_cons']=zr

In [5]:
y = clean_data['need_cons']
X = clean_data[['Road Type','Number of Lanes','Avg Daily Traffic','Operational Status', 'Bridge Length','Assessment','Lifetime']]
X_train, X_valid, y_train, y_valid = train_test_split(X, y)

In [6]:
logreg = LogisticRegression(random_state=0, solver='liblinear')
logreg.fit(X_train, y_train)
print(logreg.score(X_train, y_train))
print(logreg.score(X_valid, y_valid))

0.6739851419474662
0.6776440155208437


In [7]:
cond={'Index':[],'Road Type':[],'Number of Lanes':[],'Avg Daily Traffic':[],'Operational Status':[], 'Bridge Length':[],'Assessment':[]}

In [8]:
for t in range(1,6):
    for nl in range(2,18,4):
        for avg in range(100000,800000,200000):
            for op in range(1,6):
                for leng in range(1000,6000,1000):
                    for ass in [2,4,7]:
                            cond['Road Type'].append(t)
                            cond['Number of Lanes'].append(nl)
                            cond['Avg Daily Traffic'].append(avg)
                            cond['Operational Status'].append(op)
                            cond['Bridge Length'].append(leng)
                            cond['Assessment'].append(ass)

In [9]:
cond['Index']=np.linspace(0,len(cond['Road Type'])-1,len(cond['Road Type']),dtype=int)

In [10]:
df_cond=pd.DataFrame.from_dict(cond)
df_cond.tail()

,Index,Road Type,Number of Lanes,Avg Daily Traffic,Operational Status,Bridge Length,Assessment
5995,5995,5,14,700000,5,4000,4
5996,5996,5,14,700000,5,4000,7
5997,5997,5,14,700000,5,5000,2
5998,5998,5,14,700000,5,5000,4
5999,5999,5,14,700000,5,5000,7


In [11]:
life=pd.DataFrame(columns=list(np.arange(115)+1))
for i in range(6000):
    temp=pd.concat([pd.DataFrame(df_cond.iloc[i:i+1,1:10])]*115,ignore_index=True)
    temp['life']=np.arange(115)
    life.loc[i]=logreg.predict_proba(temp)[:,1]

In [12]:
life.head()

,1,2,3,4,5,6,7,8,9,10,...,106,107,108,109,110,111,112,113,114,115
0,0.462101,0.465228,0.468357,0.471489,0.474623,0.477759,0.480896,0.484036,0.487176,0.490318,...,0.762826,0.765093,0.767345,0.769582,0.771804,0.774011,0.776203,0.778379,0.780540,0.782686
1,0.688684,0.691373,0.694049,0.696713,0.699363,0.701999,0.704623,0.707233,0.709829,0.712412,...,0.892266,0.893468,0.894659,0.895838,0.897006,0.898161,0.899306,0.900438,0.901560,0.902670
2,0.901390,0.902502,0.903603,0.904692,0.905771,0.906838,0.907895,0.908941,0.909976,0.911001,...,0.971609,0.971954,0.972295,0.972631,0.972964,0.973293,0.973618,0.973939,0.974256,0.974569
3,0.549699,0.552809,0.555915,0.559016,0.562113,0.565206,0.568293,0.571375,0.574451,0.577521,...,0.820476,0.822321,0.824150,0.825965,0.827765,0.829550,0.831320,0.833076,0.834817,0.836543
4,0.758654,0.760949,0.763228,0.765493,0.767742,0.769976,0.772196,0.774400,0.776589,0.778762,...,0.921683,0.922586,0.923479,0.924363,0.925237,0.926102,0.926958,0.927805,0.928642,0.929471


In [13]:
result = pd.concat([df_cond.iloc[:,1:10], life], axis=1, sort=False)

In [14]:
result_index=result.set_index(['Road Type','Number of Lanes','Avg Daily Traffic','Operational Status', 'Bridge Length','Assessment'])

In [15]:
result_index.head()

1    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.462101   
                                                                             4           0.688684   
                                                                             7           0.901390   
                                                               2000          2           0.549699   
                                                                             4           0.758654   

                                                                                              2    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.465228   
                                                                             4           0.691373   
                                                                             7           0.902502   
                                                               2000          2           0.552809   
                                                                             4           0.760949   

                                                                                              3    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.468357   
                                                                             4           0.694049   
                                                                             7           0.903603   
                                                               2000          2           0.555915   
                                                                             4           0.763228   

                                                                                              4    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.471489   
                                                                             4           0.696713   
                                                                             7           0.904692   
                                                               2000          2           0.559016   
                                                                             4           0.765493   

                                                                                              5    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.474623   
                                                                             4           0.699363   
                                                                             7           0.905771   
                                                               2000          2           0.562113   
                                                                             4           0.767742   

                                                                                              6    \
Road Type Number of Lanes Avg Daily Traffic Operational Status Bridge Length Assessment             
1         2               100000            1                  1000          2           0.477759   
                                                                             4           0.701999   
                                                                             7           0.906838   
                                                 

In [16]:
result_index.to_csv('Prob_model.csv')